In [1]:
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.dates import DateFormatter, DayLocator
import plotly.graph_objects as go
import os

from IPython.display import HTML

In [2]:
# Cargar el archivo NetCDF
ds = xr.open_dataset('WRFPrecip_horario.nc')
ds

<xarray.Dataset>
Dimensions:  (Time: 1, south_north: 230, west_east: 251, time: 72)
Coordinates:
    XLAT     (Time, south_north, west_east) float32 ...
    XLONG    (Time, south_north, west_east) float32 ...
    XTIME    (time, Time) datetime64[ns] ...
Dimensions without coordinates: Time, south_north, west_east, time
Data variables:
    PRECIP   (time, Time, south_north, west_east) float32 ...

In [3]:
# Cargar el shapefile de la cuenca hidrográfica
shapefile_path = r'd:\SIG\basedata\CUENCAS\cuenca_sanjose_picadavarela.shp'
cuenca = gpd.read_file(shapefile_path)

# Extraer el nombre del shapefile sin la extensión
nombre_cuenca = os.path.splitext(os.path.basename(shapefile_path))[0]

In [4]:
# Extraer las variables de latitud y longitud
lat = ds['XLAT'].isel(Time=0)
lon = ds['XLONG'].isel(Time=0)

In [5]:
# Crear un DataArray con las coordenadas lat/lon en formato de puntos
puntos = gpd.GeoDataFrame({'geometry': gpd.points_from_xy(lon.values.flatten(), lat.values.flatten())}, crs='EPSG:4326')

In [6]:
# Filtrar los puntos que caen dentro de la cuenca hidrográfica
puntos_en_cuenca = puntos[puntos.geometry.apply(lambda x: cuenca.geometry.contains(x).any())]

In [7]:
# Reshape los índices de los puntos en la cuenca para usar en el DataArray
indices = puntos_en_cuenca.index

# Extraer los datos de precipitación para los puntos en la cuenca
precip_en_cuenca = ds['PRECIP'].isel(south_north=indices // lon.shape[1], west_east=indices % lon.shape[1])

# Calcular la precipitación areal promedio en cada paso de tiempo
precip_areal_promedio = precip_en_cuenca.mean(dim=('south_north', 'west_east'))

# Convertir a DataFrame para el hietograma
df_precip = precip_areal_promedio.to_dataframe(name='Precipitación_Areal_Promedio').reset_index()

# Calcular la lluvia acumulada
df_precip['Precipitación_Acumulada'] = df_precip['Precipitación_Areal_Promedio'].cumsum()

# Ajustar formato de las columnas a dos decimales
pd.options.display.float_format = '{:.2f}'.format
df_precip['Precipitación_Areal_Promedio'] = df_precip['Precipitación_Areal_Promedio'].round(2)
df_precip['Precipitación_Acumulada'] = df_precip['Precipitación_Acumulada'].round(2)

In [8]:
# Crear el hietograma con Plotly
fig = go.Figure()

# Gráfico de barras para la precipitación horaria
fig.add_trace(go.Bar(
    x=df_precip['XTIME'],
    y=df_precip['Precipitación_Areal_Promedio'],
    name='Precipitación Horaria',
    marker_color='blue'
))

# Línea de lluvia acumulada
fig.add_trace(go.Scatter(
    x=df_precip['XTIME'],
    y=df_precip['Precipitación_Acumulada'],
    mode='lines+markers',
    name='Precipitación Acumulada',
    line=dict(color='red')
))

# Configurar el diseño del gráfico
fig.update_layout(
    title=f'Hietograma de la Precipitación Areal Promedio en la cuenca {nombre_cuenca}',
    xaxis=dict(
        title='Tiempo',
        tickformat='%d-%b-%Y',
        dtick=86400000.0,  # Un día en milisegundos
        tickangle=0
    ),
    yaxis=dict(
        title='Precipitación Horaria (mm)',
        side='left'
    ),
    yaxis2=dict(
        title='Precipitación Acumulada (mm)',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.05, y=0.95)
)

# Mostrar el gráfico interactivo
fig.show()

In [9]:
# Mostrar el DataFrame resultante
HTML(df_precip.tail(12).to_html(index=False))

time,Time,XTIME,Precipitación_Areal_Promedio,Precipitación_Acumulada
60,0,2025-04-01 13:00:00,0.00,28.36
61,0,2025-04-01 14:00:00,0.00,28.36
62,0,2025-04-01 15:00:00,0.00,28.36
63,0,2025-04-01 16:00:00,0.00,28.36
64,0,2025-04-01 17:00:00,0.00,28.36
65,0,2025-04-01 18:00:00,0.00,28.36
66,0,2025-04-01 19:00:00,0.00,28.36
67,0,2025-04-01 20:00:00,0.00,28.36
68,0,2025-04-01 21:00:00,0.00,28.37
69,0,2025-04-01 22:00:00,0.00,28.37


In [29]:
# Establecer XTIME como el índice del DataFrame
df_precip.set_index('XTIME', inplace=True)

# Resamplear los datos a un intervalo de 3 horas y acumular las precipitaciones
df_resampled = pd.DataFrame(df_precip["Precipitación_Areal_Promedio"].resample('3H').sum())
# Calcular la lluvia acumulada
df_resampled['Precipitación_Acumulada'] = df_resampled['Precipitación_Areal_Promedio'].cumsum()

# Resetear el índice para volver a tener XTIME como columna
df_resampled.reset_index(inplace=True)

In [30]:
HTML(df_resampled.tail(48).to_html(index=False))

XTIME,Precipitación_Areal_Promedio,Precipitación_Acumulada
2025-03-29 00:00:00,0.00,0.00
2025-03-29 03:00:00,0.00,0.00
2025-03-29 06:00:00,0.00,0.00
2025-03-29 09:00:00,0.00,0.00
2025-03-29 12:00:00,0.00,0.00
2025-03-29 15:00:00,0.00,0.00
2025-03-29 18:00:00,0.00,0.00
2025-03-29 21:00:00,0.00,0.00
2025-03-30 00:00:00,0.00,0.00
2025-03-30 03:00:00,0.00,0.00
